In [1]:
import json

# set the subset of the data to use
subset = 1
# load the data
data = json.load(open('../data/subsets/visdial_1.0_train_' +
                 str(subset) + 'percent_subset.json'))['data']
# load the dialogs
dialogs = data['dialogs']

In [2]:
import pickle

'''Load the GAP output of the History Graphs Batch'''
with open('../embeddings/history/'+ str(subset) +'/history_batch_GAP.pkl', 'rb') as f:
    history_batch_GAP = pickle.load(f)

'''Load the GAP output of the Question Graphs'''
with open('../embeddings/questions/'+ str(subset) +'/question_GAP.pkl', 'rb') as f:
    question_GAP = pickle.load(f)

'''Load the GAP output of the Image Graphs'''
with open('../embeddings/images/instance/'+ str(subset) +'/image_GAP.pkl', 'rb') as f:
    image_GAP = pickle.load(f)

'''Load the Answer Embeddings'''
with open('../embeddings/answers/' + str(subset) + '/answer_embeddings.pkl', 'rb') as f:
    answer_embeddings = []
    while True:
        try:
            answer_embeddings.append(pickle.load(f))
        except EOFError:
            break

In [3]:
'''For each <dialog> entry, we obtain the:
    1. 10 questions from the <dialog rounds>
    2. Copy the image 10 times from the <dialog rounds> since the image is the same for all rounds'''
from tqdm import tqdm

question_batch_GAP = []
image_batch_GAP = []

for dialog in tqdm(dialogs):
    # get the image (the index of the image is the same as the index of the dialog); what the image ID contains is the reference to the image from COCO
    image_id_dialog = image_GAP[dialogs.index(dialog)]
    # copy the image 10 times
    image_id_dialog = [image_id_dialog] * 10
    # append the image to the image batch
    image_batch_GAP.append(image_id_dialog)

    # question gap for this dialog
    question_GAP_dialog = []
    # get the questions
    for round in dialog['dialog']:
        # get the question
        question_GAP_dialog.append(question_GAP[round['question']])
    # append the questions to the question batch
    question_batch_GAP.append(question_GAP_dialog)

'''Save the question GAP embeddings'''
with open('../embeddings/fusion/question_GAP_batch_control' + str(subset) + '.pkl', 'wb') as f:
    pickle.dump(question_batch_GAP, f)

'''Save the image GAP embeddings'''
with open('../embeddings/fusion/image_GAP_batch_control' + str(subset) + '.pkl', 'wb') as f:
    pickle.dump(image_batch_GAP, f)

100%|██████████| 1233/1233 [00:00<00:00, 10989.47it/s]
